# Module 3a: Local Agent Observability with OpenTelemetry

In this module, you'll add observability to your **local** agent using AWS OpenTelemetry and GenAI semantic conventions for CloudWatch GenAI Observability Dashboard.

## What You'll Learn

- How to enhance your Module 1 agent with OpenTelemetry instrumentation
- How to create custom spans using GenAI semantic conventions
- How to view traces in CloudWatch GenAI Observability Dashboard

## GenAI Semantic Conventions

For spans to appear correctly in CloudWatch GenAI Observability Dashboard, they must use:

1. **Recognized tracer scopes**: `openinference.instrumentation.*`, `strands.telemetry.tracer`, etc.
2. **GenAI semantic convention attributes**:
   - `gen_ai.operation.name`: "invoke_agent" or "execute_tool"
   - `gen_ai.agent.name`: Agent identifier
   - `session.id`: Required for trace correlation
   - `gen_ai.usage.input_tokens`, `gen_ai.usage.output_tokens`: Token metrics

## Architecture

```
┌─────────────────────────────────────────────────────────────────────────┐
│        skills_agent_observable.py + GenAI Semantic Conventions          │
│  ┌─────────────────────────────────────────────────────────────────┐    │
│  │  opentelemetry-instrument python agent/skills_agent_observable.py│    │
│  │                                                                 │    │
│  │  Tracer scope: openinference.instrumentation.claude_agent_sdk   │    │
│  │                                                                 │    │
│  │  Custom spans (GenAI semantic conventions):                     │    │
│  │  └── invoke_agent student-analytics-agent                       │    │
│  │      ├── gen_ai.operation.name: "invoke_agent"                  │    │
│  │      ├── gen_ai.agent.name, session.id                          │    │
│  │      ├── gen_ai.usage.input_tokens, output_tokens               │    │
│  │      └── execute_tool <tool_name> (child spans)                 │    │
│  │          └── gen_ai.tool.name, db.statement (for SQL)           │    │
│  └─────────────────────────────────────────────────────────────────┘    │
└─────────────────────────────────────────────────────────────────────────┘
                                     │ OTLP/HTTP
                                     ▼
┌─────────────────────────────────────────────────────────────────────────┐
│                    Amazon CloudWatch                                    │
│           ┌──────────────────────────────────────────────────────┐      │
│           │  GenAI Observability Dashboard                       │      │
│           │  - Agent sessions & workflows                        │      │
│           │  - Tool usage & skill loading                        │      │
│           │  - Token usage (input/output)                        │      │
│           │  - Cost tracking per session                         │      │
│           └──────────────────────────────────────────────────────┘      │
└─────────────────────────────────────────────────────────────────────────┘
```

## Prerequisites

- **Module 1 completed** - Your agent must be working
- AWS credentials configured
- Transaction Search enabled in CloudWatch

## Step 1: Setup Environment

In [ ]:
import os
import sys
import json
import boto3
from pathlib import Path
from dotenv import load_dotenv

# Set up paths
workshop_root = Path("..").resolve()
sys.path.insert(0, str(workshop_root))

# Load environment
load_dotenv(workshop_root / ".env")

# Ensure Bedrock is used
os.environ["CLAUDE_CODE_USE_BEDROCK"] = "1"

region = os.getenv('AWS_REGION', 'us-east-1')
print(f"Workshop root: {workshop_root}")
print(f"AWS Region: {region}")


Workshop root: /Users/mmelli/Documents/projects/claude-code-agentcore/agentic-ai-with-claude-agent-sdk-and-amazon-bedrock-agentcore
AWS Region: us-west-2


## Step 2: Install OpenTelemetry Dependencies

The `aws-opentelemetry-distro` automatically instruments:
- **Amazon Bedrock** - Model invocations, tokens, latency
- **HTTP requests** - External API calls
- **boto3 calls** - AWS SDK operations (Athena, S3, etc.)

The `openinference-instrumentation-bedrock` provides:
- **GenAI semantic conventions** - Proper span attributes for CloudWatch GenAI Observability
- **Automatic Bedrock instrumentation** - Auto-creates spans for Bedrock API calls

In [ ]:
!pip install opentelemetry-api opentelemetry-sdk aws-opentelemetry-distro --quiet
!pip install openinference-instrumentation-bedrock openinference-semantic-conventions --quiet

print("Installed packages:")
!pip show aws-opentelemetry-distro openinference-instrumentation-bedrock | grep -E "Name|Version"

Installed packages:
Name: aws-opentelemetry-distro
Version: 0.12.2
Name: openinference-instrumentation-bedrock
Version: 0.1.32


## Step 3: Enable Transaction Search

Transaction Search enables span visibility in the `aws/spans` log group.

**Important**: You must enable Transaction Search before traces will appear in CloudWatch. This step should have been done in lab0, if you haven't enabled the Transaction Search, please make sure you wait for enough time before you can see the trace records.

In [3]:
import subprocess

print("Checking Transaction Search status...")
print("=" * 60)

try:
    result = subprocess.run(
        ['aws', 'xray', 'get-trace-segment-destination', '--region', region],
        capture_output=True, text=True
    )
    
    if result.returncode == 0:
        response = json.loads(result.stdout)
        destination = response.get('Destination', 'Unknown')
        status = response.get('Status', 'Unknown')
        
        print(f"Destination: {destination}")
        print(f"Status: {status}")
        
        if destination == 'CloudWatchLogs' and status == 'ACTIVE':
            print("\n✅ Transaction Search is ENABLED!")
        else:
            print("\n⚠️  Transaction Search needs to be enabled!")
            print(f"\nOpen this link to enable:")
            print(f"https://{region}.console.aws.amazon.com/cloudwatch/home?region={region}#xray:settings/transaction-search")
            print("\nSteps:")
            print("1. Click 'Edit'")
            print("2. Select 'Ingest spans as structured logs'")
            print("3. Click 'Save'")
            print("4. Wait ~10 minutes for it to become active")
except Exception as e:
    print(f"Error checking status: {e}")
    print(f"\nManually enable at:")
    print(f"https://{region}.console.aws.amazon.com/cloudwatch/home?region={region}#xray:settings/transaction-search")

Checking Transaction Search status...
Destination: CloudWatchLogs
Status: ACTIVE

✅ Transaction Search is ENABLED!


## Step 4: Create CloudWatch Log Group

In [4]:
logs_client = boto3.client('logs', region_name=region)

agent_name = 'student-analytics-agent'
agent_version = '1.0.0'
log_group_name = f'agents/{agent_name}'
log_stream_name = 'default'

print("Setting up CloudWatch Log Group...")
print("=" * 60)

try:
    logs_client.create_log_group(logGroupName=log_group_name)
    print(f"✅ Created log group: {log_group_name}")
except logs_client.exceptions.ResourceAlreadyExistsException:
    print(f"✅ Log group exists: {log_group_name}")

try:
    logs_client.create_log_stream(logGroupName=log_group_name, logStreamName=log_stream_name)
    print(f"✅ Created log stream: {log_stream_name}")
except logs_client.exceptions.ResourceAlreadyExistsException:
    print(f"✅ Log stream exists: {log_stream_name}")

Setting up CloudWatch Log Group...
✅ Log group exists: agents/student-analytics-agent
✅ Log stream exists: default


## Step 5: Configure OpenTelemetry Environment

| Variable | Purpose |
|----------|--------|
| `OTEL_PYTHON_DISTRO` | Use AWS Distro for OpenTelemetry |
| `OTEL_RESOURCE_ATTRIBUTES` | Service identification (agent name, version) |
| `AGENT_OBSERVABILITY_ENABLED` | Enable custom span creation in agent code |

In [5]:
# Create .env.observability file
otel_env_content = f"""# AWS OpenTelemetry Configuration
OTEL_RESOURCE_ATTRIBUTES=service.name={agent_name},service.version={agent_version}
OTEL_PYTHON_DISTRO=aws_distro
OTEL_PYTHON_CONFIGURATOR=aws_configurator
OTEL_EXPORTER_OTLP_PROTOCOL=http/protobuf
OTEL_TRACES_EXPORTER=otlp
OTEL_EXPORTER_OTLP_LOGS_HEADERS=x-aws-log-group={log_group_name},x-aws-log-stream={log_stream_name},x-aws-metric-namespace=student-analytics
AGENT_OBSERVABILITY_ENABLED=true
"""

otel_env_path = workshop_root / ".env.observability"
otel_env_path.write_text(otel_env_content)
print(f"Created: {otel_env_path}")
print(otel_env_content)

Created: /Users/mmelli/Documents/projects/claude-code-agentcore/agentic-ai-with-claude-agent-sdk-and-amazon-bedrock-agentcore/.env.observability
# AWS OpenTelemetry Configuration
OTEL_RESOURCE_ATTRIBUTES=service.name=student-analytics-agent,service.version=1.0.0
OTEL_PYTHON_DISTRO=aws_distro
OTEL_PYTHON_CONFIGURATOR=aws_configurator
OTEL_EXPORTER_OTLP_PROTOCOL=http/protobuf
OTEL_TRACES_EXPORTER=otlp
OTEL_EXPORTER_OTLP_LOGS_HEADERS=x-aws-log-group=agents/student-analytics-agent,x-aws-log-stream=default,x-aws-metric-namespace=student-analytics
AGENT_OBSERVABILITY_ENABLED=true



## Step 6: Create Observable Agent

We'll create an enhanced version of the Module 1 agent with OpenTelemetry instrumentation.

**Key additions:**
- OpenTelemetry tracer initialization
- Session context in baggage for correlation
- `agent_session` root span with agent attributes
- Child spans for each tool call (Skill, Read, Bash)
- Completion metrics (cost, turns, duration)

In [6]:
# Check if observable agent exists
observable_agent = workshop_root / "agent" / "skills_agent_observable.py"

if observable_agent.exists():
    print(f"✅ Observable agent exists: {observable_agent}")
    print("\nKey differences from skills_agent.py:")
    print("  - OpenTelemetry imports and tracer initialization")
    print("  - Session ID argument for trace correlation")
    print("  - agent_session root span with attributes")
    print("  - Child spans for tool calls")
    print("  - Cost, turns, and duration metrics")
else:
    print(f"❌ Observable agent not found at: {observable_agent}")
    print("\nPlease ensure skills_agent_observable.py exists in the agent/ directory.")

✅ Observable agent exists: /Users/mmelli/Documents/projects/claude-code-agentcore/agentic-ai-with-claude-agent-sdk-and-amazon-bedrock-agentcore/agent/skills_agent_observable.py

Key differences from skills_agent.py:
  - OpenTelemetry imports and tracer initialization
  - Session ID argument for trace correlation
  - agent_session root span with attributes
  - Child spans for tool calls
  - Cost, turns, and duration metrics


## Step 7: Understanding GenAI Semantic Convention Spans

### Required for CloudWatch GenAI Observability

For spans to appear in CloudWatch GenAI Observability Dashboard, you must use:

1. **Recognized tracer scope**: `openinference.instrumentation.claude_agent_sdk`
2. **GenAI semantic convention attributes** on spans

### Span Naming Convention

| Operation | Span Name Format |
|-----------|-----------------|
| Agent invocation | `invoke_agent {agent_name}` |
| Tool execution | `execute_tool {tool_name}` |

### Attributes on `invoke_agent` span

| Attribute | Description |
|-----------|-------------|
| `gen_ai.operation.name` | "invoke_agent" |
| `gen_ai.system` | "claude-agent-sdk" |
| `gen_ai.agent.name` | Agent identifier |
| `gen_ai.prompt` | User query (truncated) |
| `gen_ai.agent.tools` | JSON array of available tools |
| `session.id` | Session identifier for correlation |
| `gen_ai.usage.input_tokens` | Input token count |
| `gen_ai.usage.output_tokens` | Output token count |

### Attributes on `execute_tool` span

| Attribute | Description |
|-----------|-------------|
| `gen_ai.operation.name` | "execute_tool" |
| `gen_ai.tool.name` | Name of the tool |
| `tool.sequence` | Order of tool call |
| `db.statement` | SQL query (for Athena tool) |
| `db.system` | "athena" (for Athena tool) |

### Events (GenAI Semantic Convention)

| Event Name | Purpose |
|------------|---------|
| `gen_ai.user.message` | User query submitted |
| `gen_ai.assistant.message` | Assistant response |
| `gen_ai.agent.finish` | Agent completed successfully |
| `gen_ai.agent.error` | Agent encountered error |

```
Trace Hierarchy:
└── invoke_agent student-analytics-agent
    ├── gen_ai.operation.name: "invoke_agent"
    ├── gen_ai.agent.name: "student-analytics-agent"
    ├── session.id: "workshop-abc123"
    ├── gen_ai.usage.input_tokens: 1500
    ├── gen_ai.usage.output_tokens: 800
    │
    ├── execute_tool Skill
    │   ├── gen_ai.tool.name: "Skill"
    │   └── skill.name: "enrollment-analytics"
    │
    └── execute_tool mcp__athena__execute_athena_query
        ├── gen_ai.tool.name: "mcp__athena__execute_athena_query"
        ├── db.system: "athena"
        └── db.statement: "SELECT COUNT(...)..."
```

## Step 8: Run Observable Agent

Run the observable agent with `opentelemetry-instrument`:

```bash
opentelemetry-instrument python agent/skills_agent_observable.py --session-id demo-001 "Your query"
```

In [ ]:
import subprocess
import uuid

test_query = "How many students are currently enrolled?"
session_id = f"workshop-{uuid.uuid4().hex[:8]}"

print("Running Observable Agent with OpenTelemetry instrumentation...")
print("=" * 70)
print(f"Agent: {observable_agent}")
print(f"Session ID: {session_id}")
print(f"Query: {test_query}")
print("=" * 70 + "\n")

# Build command with session ID
cmd = [
    "opentelemetry-instrument",
    sys.executable, str(observable_agent),
    "--session-id", session_id,
    test_query
]

# Set up OTEL environment
env = os.environ.copy()
env.update({
    "OTEL_PYTHON_DISTRO": "aws_distro",
    "OTEL_PYTHON_CONFIGURATOR": "aws_configurator",
    "OTEL_EXPORTER_OTLP_PROTOCOL": "http/protobuf",
    "OTEL_TRACES_EXPORTER": "otlp",
    "OTEL_EXPORTER_OTLP_LOGS_HEADERS": f"x-aws-log-group={log_group_name},x-aws-log-stream={log_stream_name},x-aws-metric-namespace=student-analytics",
    "OTEL_RESOURCE_ATTRIBUTES": f"service.name={agent_name},service.version={agent_version}",
    "AGENT_OBSERVABILITY_ENABLED": "true",
    "CLAUDE_CODE_USE_BEDROCK": "1"
})

try:
    result = subprocess.run(
        cmd, 
        cwd=str(workshop_root),
        env=env, 
        capture_output=True, 
        text=True, 
        timeout=300
    )
    
    # Show output
    if result.stdout:
        print(result.stdout[:5000])
    
    if result.returncode == 0:
        print("\n" + "=" * 70)
        print("✅ Agent executed with observability!")
        print(f"\nSession ID: {session_id}")
        print(f"\n⏰ Traces will be available in CloudWatch within 1-2 minutes.")
    else:
        print(f"\n❌ Error (exit code {result.returncode}):")
        print(result.stderr[:2000] if result.stderr else "(no stderr)")
        
except subprocess.TimeoutExpired:
    print("⚠️ Timed out after 5 minutes")
except FileNotFoundError:
    print("❌ opentelemetry-instrument not found. Run: pip install aws-opentelemetry-distro")

Running Observable Agent with OpenTelemetry instrumentation...
Agent: /Users/mmelli/Documents/projects/claude-code-agentcore/agentic-ai-with-claude-agent-sdk-and-amazon-bedrock-agentcore/agent/skills_agent_observable.py
Session ID: workshop-a9a12f91
Query: How many students are currently enrolled?

[Observability] Session: workshop-a9a12f91
[Observability] OpenInference Bedrock instrumentation: enabled
Loaded project context from CLAUDE.md (4802 chars)
OBSERVABLE STUDENT ANALYTICS AGENT (With OpenTelemetry)
Request ID: 5ff1410a-4c3c-4b98-b234-63ec3f3fcdf9

User Query: How many students are currently enrolled?

--------------------------------------------------------------------------------
I'll help you find the number of currently enrolled students. Let me start by loading the appropriate skill and reviewing the table documentation.

[Loading skill: enrollment]
Base directory for this skill: /Users/mmelli/Documents/projects/claude-code-agentcore/agentic-ai-with-claude-agent-sdk-and-am

## Step 9: View Traces in CloudWatch

In [8]:
print("CloudWatch Observability Links")
print("=" * 70)

base_url = f"https://{region}.console.aws.amazon.com/cloudwatch"

print(f"\n📊 GenAI Observability Dashboard (Agents):")
print(f"   {base_url}/home?region={region}#/gen-ai-observability/agent-core/agents")


print("\n" + "=" * 70)
print("\n📌 How to find your traces:")
print(f"   1. Open GenAI Observability Dashboard link above")
print(f"   2. Look for service.name = '{agent_name}'")
print(f"   3. Click on a session to see the trace waterfall")

CloudWatch Observability Links

📊 GenAI Observability Dashboard (Agents):
   https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#/gen-ai-observability/agent-core/agents


📌 How to find your traces:
   1. Open GenAI Observability Dashboard link above
   2. Look for service.name = 'student-analytics-agent'
   3. Click on a session to see the trace waterfall


## Step 10: GenAI Semantic Convention Attributes Reference

### Agent Invocation Span (`invoke_agent {agent_name}`)

| Attribute | Type | Description |
|-----------|------|-------------|
| `gen_ai.operation.name` | string | "invoke_agent" |
| `gen_ai.system` | string | "claude-agent-sdk" |
| `gen_ai.agent.name` | string | Agent identifier |
| `gen_ai.prompt` | string | User query (first 500 chars) |
| `gen_ai.agent.tools` | string | JSON array of tool names |
| `session.id` | string | Session ID for correlation |
| `gen_ai.usage.input_tokens` | int | Input tokens used |
| `gen_ai.usage.output_tokens` | int | Output tokens used |
| `gen_ai.usage.total_tokens` | int | Total tokens used |
| `agent.cost_usd` | float | Estimated cost in USD |
| `agent.num_turns` | int | Number of conversation turns |
| `agent.duration_ms` | int | Total execution time |

### Tool Execution Span (`execute_tool {tool_name}`)

| Attribute | Type | Description |
|-----------|------|-------------|
| `gen_ai.operation.name` | string | "execute_tool" |
| `gen_ai.tool.name` | string | Tool name |
| `tool.sequence` | int | Order in which tool was called |
| `session.id` | string | Session ID |
| `skill.name` | string | Skill name (for Skill tool) |
| `file.path` | string | File path (for Read/Write tools) |
| `db.system` | string | "athena" (for Athena tool) |
| `db.statement` | string | SQL query (for Athena tool) |

### Events

| Event | Attributes |
|-------|------------|
| `gen_ai.user.message` | `content`, `session.id` |
| `gen_ai.assistant.message` | `content_preview`, `session.id` |
| `gen_ai.agent.finish` | `success`, `duration_ms`, `cost_usd`, `num_turns` |
| `gen_ai.agent.error` | `error.message`, `error.type`, `session.id` |
| `skill_loaded` | `skill`, `session.id` |
| `sql_query_submitted` | `sql_preview`, `session.id` |

## Step 11: Run Additional Queries

Run several queries with different session IDs to see multiple traces in the dashboard.

In [ ]:
test_queries = [
    ("What is the average GPA across all students?", "academic"),
    ("Which department has the highest enrollment?", "enrollment"),
]

print("Running additional queries for trace analysis...")
print("=" * 70)

session_ids = []

for i, (query_text, query_type) in enumerate(test_queries, 1):
    session_id = f"workshop-{query_type}-{uuid.uuid4().hex[:6]}"
    session_ids.append(session_id)
    
    print(f"\nQuery {i}: {query_text}")
    print(f"Session: {session_id}")
    print("-" * 50)
    
    cmd = [
        "opentelemetry-instrument", 
        sys.executable, str(observable_agent),
        "--session-id", session_id,
        query_text
    ]
    
    try:
        result = subprocess.run(
            cmd, cwd=str(workshop_root), env=env, 
            capture_output=True, text=True, timeout=300
        )
        
        if result.returncode == 0:
            # Show just the summary
            lines = result.stdout.split('\n')
            for line in lines[-15:]:
                if line.strip():
                    print(line)
            print("✅ Completed")
        else:
            print(f"❌ Error: {result.stderr[:300]}")
            
    except subprocess.TimeoutExpired:
        print("⚠️ Timed out")

print("\n" + "=" * 70)
print(f"✅ All queries completed.")
print(f"\nSession IDs for filtering:")
for sid in session_ids:
    print(f"  - {sid}")

## Cleanup (Optional)

## Key Takeaways

### What You Learned

1. **GenAI Semantic Conventions** - Created spans using OpenTelemetry GenAI semantic conventions
2. **Recognized Tracer Scopes** - Use `openinference.instrumentation.*` for CloudWatch compatibility
3. **Session Tracking** - Use `session.id` attribute for trace correlation
4. **Token Metrics** - Track `gen_ai.usage.input_tokens` and `output_tokens`
5. **Structured Events** - Use `gen_ai.user.message`, `gen_ai.assistant.message`, `gen_ai.agent.finish`
6. **CloudWatch Integration** - View everything in GenAI Observability Dashboard

### Running Your Agent with Observability

```bash
# From workshop root directory:
opentelemetry-instrument python agent/skills_agent_observable.py --session-id user-123 "Your query"
```

### Key GenAI Semantic Convention Attributes

| Purpose | Attribute |
|---------|-----------|
| Operation type | `gen_ai.operation.name` ("invoke_agent" or "execute_tool") |
| Agent name | `gen_ai.agent.name` |
| Session correlation | `session.id` |
| Input tokens | `gen_ai.usage.input_tokens` |
| Output tokens | `gen_ai.usage.output_tokens` |
| Tool name | `gen_ai.tool.name` |
| SQL query | `db.statement` |

### Important Notes

- **Tracer scope matters**: CloudWatch GenAI Observability only recognizes certain scopes
- **Session ID required**: Include `session.id` on all spans and events for proper correlation
- **Events with structure**: Use `gen_ai.event.name` attribute in event data

---

*Workshop: Build Agentic AI Applications with Claude Agent SDK and Amazon Bedrock AgentCore*